Imports

In [38]:
import utils.scrape_no_js as bourso
import utils.scrape_js as euronext
import utils.inputs as inputs
import utils.helper_functions as hf

from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
import yfinance as yf
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)

Config

In [2]:
buy_time = 'Open'
loss_limit = 0.997

cumulated_probs_target = 0.85

fee = 0.002

download_days = 500 #should be > 151 = 90 (input days) * 7/5 (days per week) * 1.2 (margin in case of vacations)

target_buy_amount = 2100

base_url_euronext = "https://live.euronext.com/en/product/equities/"
base_url_bourso = "https://www.boursorama.com/cours/"
base_url_fortuneo = "https://bourse.fortuneo.fr/actions/"

model_path = './outputs/classifier_model.keras'
scaler_path = './outputs/scaler.pkl'

In [3]:
tickers_file_path = './db/tickers_euronext_regulated_euro_500k€.xlsx'
df_tickers = pd.read_excel(tickers_file_path)

num_tickers = len(df_tickers)
print(f'number of tickers: {num_tickers}')

number of tickers: 854


Download past data

In [4]:
def get_past_date(days_past):
    today = date.today()
    past_date = today - timedelta(days=days_past)
    formatted_date = past_date.strftime('%Y-%m-%d')
    return formatted_date

def days_difference(date1, date2): # can handle date objects and 'YYYY-MM-DD' strings
    date1 = pd.to_datetime(date1)
    date2 = pd.to_datetime(date2)

    difference = abs((date2 - date1).days)
    return difference

In [5]:
start_date = get_past_date(download_days)
today = get_past_date(0)

print(f'start download date: {start_date}')
print(f'today: {today}')

start download date: 2023-03-13
today: 2024-07-25


In [6]:
yahoo_tickers = df_tickers['yahoo'].iloc[1:num_tickers+1].values.tolist()

download_past_data = yf.download(yahoo_tickers, start=start_date, end=today)
df_download_past = pd.DataFrame(download_past_data)
df_download_past_filled = hf.fillnavalues(df_download_past)
# df_download = df_download.dropna(axis='columns', how='all') #remove columns with only nan values

print(df_download_past_filled.tail(3).to_markdown())


[*********************100%%**********************]  853 of 853 completed

3 Failed downloads:
['BNJW.AS', 'NAIW.AS', 'WTFIN.MI']: Exception('%ticker%: No timezone found, symbol may be delisted')


| Date                |   ('Adj Close', 'A2A.MI') |   ('Adj Close', 'AAA.PA') |   ('Adj Close', 'AALB.AS') |   ('Adj Close', 'AB.PA') |   ('Adj Close', 'ABCA.PA') |   ('Adj Close', 'ABEO.PA') |   ('Adj Close', 'ABI.BR') |   ('Adj Close', 'ABN.AS') |   ('Adj Close', 'ABNX.PA') |   ('Adj Close', 'ABO.BR') |   ('Adj Close', 'ABT.MI') |   ('Adj Close', 'ABVX.PA') |   ('Adj Close', 'AC.PA') |   ('Adj Close', 'AC5.MI') |   ('Adj Close', 'ACA.PA') |   ('Adj Close', 'ACAN.PA') |   ('Adj Close', 'ACCB.BR') |   ('Adj Close', 'ACE.MI') |   ('Adj Close', 'ACKB.BR') |   ('Adj Close', 'AD.AS') |   ('Adj Close', 'ADB.MI') |   ('Adj Close', 'ADOC.PA') |   ('Adj Close', 'ADP.PA') |   ('Adj Close', 'ADYEN.AS') |   ('Adj Close', 'AED.BR') |   ('Adj Close', 'AEDES.MI') |   ('Adj Close', 'AEF.MI') |   ('Adj Close', 'AELIS.PA') |   ('Adj Close', 'AF.PA') |   ('Adj Close', 'AFME.PA') |   ('Adj Close', 'AGFB.BR') |   ('Adj Close', 'AGN.AS') |   ('Adj Close', 'AGS.BR') |   ('Adj Close', 'AI.PA') |   ('Adj Clos

In [7]:
last_index = df_download_past_filled.index[-1]
last_download_past_date = last_index.date()
print(f'last download date: {last_download_past_date}')

download_missing_days = days_difference(today, last_download_past_date)
print(f'download missing days: {download_missing_days}')

# if download_missing_days > 1:
#     raise ValueError("Too many download missing days")


last download date: 2024-07-24
download missing days: 1


Download today's data

In [8]:
bourso_tickers = df_tickers['bourso'].iloc[1:num_tickers+1].values.tolist()

df_download_today = bourso.get_theoretical_opening_prices(bourso_tickers)

print(df_download_today)

         ticker  bourso_theor_open_price
0        1rAAGN                   5.8500
1       1rAAJAX                  10.2500
2      1rAAMUND                   1.2700
3        1rAALX                   1.2500
4       1rAAZRN                   1.6000
5      1rAAZRNW                      NaN
6       1rABSGR                   4.7200
7        1rABNJ                   8.9000
8       1rABNJW                      NaN
9       1rABESI                 140.6500
10       1rABAI                   0.7700
11      1rABAIW                      NaN
12      1rABGHL                  27.0000
13      1rABRNL                  10.2000
14     1rACABKA                   3.6600
15     1rACMCOM                   7.7900
16      1rACRBN                  20.3000
17      1rACTAC                   3.0700
18       1rACVC                  17.5000
19       1rADGB                   0.6700
20     1rAEAS2P                   0.5700
21      1rAEBUS                   1.2670
22      1rAEHCS                   9.1500
23      1rAENVI 

Transform past data

In [9]:
def get_single_level_df(df, ohlcv):
    new_df = df[[ohlcv]]
    new_df = hf.remove_top_column_name(new_df)

    return new_df

def get_ohlcv_dfs(df):
    df_open = get_single_level_df(df, 'Open')
    df_high = get_single_level_df(df, 'High')
    df_low = get_single_level_df(df, 'Low')
    df_close = get_single_level_df(df, 'Close')
    df_volume = get_single_level_df(df, 'Volume')
    
    return {'df_open': df_open, 'df_high': df_high, 'df_low': df_low,
            'df_close': df_close, 'df_volume': df_volume}

In [10]:
list_dfs_ohlcv = get_ohlcv_dfs(df_download_past_filled)
df_transformed_past = get_single_level_df(df_download_past_filled, buy_time)

print(df_transformed_past.tail(3).to_markdown())
print(df_transformed_past.shape)

| Date                |   A2A.MI |   AAA.PA |   AALB.AS |   AB.PA |   ABCA.PA |   ABEO.PA |   ABI.BR |   ABN.AS |   ABNX.PA |   ABO.BR |   ABT.MI |   ABVX.PA |   AC.PA |   AC5.MI |   ACA.PA |   ACAN.PA |   ACCB.BR |   ACE.MI |   ACKB.BR |   AD.AS |   ADB.MI |   ADOC.PA |   ADP.PA |   ADYEN.AS |   AED.BR |   AEDES.MI |   AEF.MI |   AELIS.PA |   AF.PA |   AFME.PA |   AGFB.BR |   AGN.AS |   AGS.BR |   AI.PA |   AIR.PA |   AJAX.AS |   AKE.PA |   AKOM.PA |   AKW.PA |   AKZA.AS |   ALFEN.AS |   ALK.MI |   ALLFG.AS |   ALO.PA |   ALTA.PA |   ALTR.LS |   ALW.MI |   ALX.AS |   AM.PA |   AMG.AS |   AMP.MI |   AMPLI.PA |   AMUN.PA |   AMUND.AS |   ANIM.MI |   ANTIN.PA |   APAM.AS |   ARAMI.PA |   ARCAD.AS |   AREIT.PA |   ARG.PA |   ARGX.BR |   ARIS.MI |   ARN.MI |   ARTE.PA |   ARTO.PA |   ARVBS.PA |   ARVEN.PA |   ASC.MI |   ASCE.BR |   ASM.AS |   ASML.AS |   ASRNL.AS |   ASY.PA |   ATE.PA |   ATEB.BR |   ATEME.PA |   ATLD.PA |   ATO.PA |   AUB.PA |   AUGR.PA |   AURE.PA |   AUTME.MI |   AV.MI 

Transform today's data

In [11]:
mapping_dict = dict(zip(df_tickers['bourso'], df_tickers['yahoo']))

df_download_today['ticker'] = df_download_today['ticker'].map(mapping_dict)

print(df_download_today)


       ticker  bourso_theor_open_price
0      AGN.AS                   5.8500
1     AJAX.AS                  10.2500
2    AMUND.AS                   1.2700
3      ALX.AS                   1.2500
4     AZRN.AS                   1.6000
5    AZRNW.AS                      NaN
6     BSGR.AS                   4.7200
7      BNJ.AS                   8.9000
8     BNJW.AS                      NaN
9     BESI.AS                 140.6500
10     BAI.AS                   0.7700
11    BAIW.AS                      NaN
12    BGHL.AS                  27.0000
13    BRNL.AS                  10.2000
14   CABKA.AS                   3.6600
15   CMCOM.AS                   7.7900
16    CRBN.AS                  20.3000
17    CTAC.AS                   3.0700
18     CVC.AS                  17.5000
19     DGB.AS                   0.6700
20   EAS2P.AS                   0.5700
21    EBUS.AS                   1.2670
22    EHCS.AS                   9.1500
23    ENVI.AS                   5.5000
24     ECT.AS            

In [12]:
df_transformed_today = df_download_today.set_index('ticker').T

today_date = datetime.now().strftime("%Y-%m-%d 00:00:00")
df_transformed_today.index = [today_date]

df_transformed_today = df_transformed_today.reindex(sorted(df_transformed_today.columns), axis='columns')
df_transformed_today = df_transformed_today.rename_axis('Date')

print(df_transformed_today.to_markdown())
print(df_transformed_today.shape)

| Date                |   A2A.MI |   AAA.PA |   AALB.AS |   AB.PA |   ABCA.PA |   ABEO.PA |   ABI.BR |   ABN.AS |   ABNX.PA |   ABO.BR |   ABT.MI |   ABVX.PA |   AC.PA |   AC5.MI |   ACA.PA |   ACAN.PA |   ACCB.BR |   ACE.MI |   ACKB.BR |   AD.AS |   ADB.MI |   ADOC.PA |   ADP.PA |   ADYEN.AS |   AED.BR |   AEDES.MI |   AEF.MI |   AELIS.PA |   AF.PA |   AFME.PA |   AGFB.BR |   AGN.AS |   AGS.BR |   AI.PA |   AIR.PA |   AJAX.AS |   AKE.PA |   AKOM.PA |   AKW.PA |   AKZA.AS |   ALFEN.AS |   ALK.MI |   ALLFG.AS |   ALO.PA |   ALTA.PA |   ALTR.LS |   ALW.MI |   ALX.AS |   AM.PA |   AMG.AS |   AMP.MI |   AMPLI.PA |   AMUN.PA |   AMUND.AS |   ANIM.MI |   ANTIN.PA |   APAM.AS |   ARAMI.PA |   ARCAD.AS |   AREIT.PA |   ARG.PA |   ARGX.BR |   ARIS.MI |   ARN.MI |   ARTE.PA |   ARTO.PA |   ARVBS.PA |   ARVEN.PA |   ASC.MI |   ASCE.BR |   ASM.AS |   ASML.AS |   ASRNL.AS |   ASY.PA |   ATE.PA |   ATEB.BR |   ATEME.PA |   ATLD.PA |   ATO.PA |   AUB.PA |   AUGR.PA |   AURE.PA |   AUTME.MI |   AV.MI 

Transform: Assemble past and today's data

In [13]:
df_transformed = pd.concat([df_transformed_past, df_transformed_today], axis=0)

print(df_transformed.tail(3).to_markdown())

| Date                |   A2A.MI |   AAA.PA |   AALB.AS |   AB.PA |   ABCA.PA |   ABEO.PA |   ABI.BR |   ABN.AS |   ABNX.PA |   ABO.BR |   ABT.MI |   ABVX.PA |   AC.PA |   AC5.MI |   ACA.PA |   ACAN.PA |   ACCB.BR |   ACE.MI |   ACKB.BR |   AD.AS |   ADB.MI |   ADOC.PA |   ADP.PA |   ADYEN.AS |   AED.BR |   AEDES.MI |   AEF.MI |   AELIS.PA |   AF.PA |   AFME.PA |   AGFB.BR |   AGN.AS |   AGS.BR |   AI.PA |   AIR.PA |   AJAX.AS |   AKE.PA |   AKOM.PA |   AKW.PA |   AKZA.AS |   ALFEN.AS |   ALK.MI |   ALLFG.AS |   ALO.PA |   ALTA.PA |   ALTR.LS |   ALW.MI |   ALX.AS |   AM.PA |   AMG.AS |   AMP.MI |   AMPLI.PA |   AMUN.PA |   AMUND.AS |   ANIM.MI |   ANTIN.PA |   APAM.AS |   ARAMI.PA |   ARCAD.AS |   AREIT.PA |   ARG.PA |   ARGX.BR |   ARIS.MI |   ARN.MI |   ARTE.PA |   ARTO.PA |   ARVBS.PA |   ARVEN.PA |   ASC.MI |   ASCE.BR |   ASM.AS |   ASML.AS |   ASRNL.AS |   ASY.PA |   ATE.PA |   ATEB.BR |   ATEME.PA |   ATLD.PA |   ATO.PA |   AUB.PA |   AUGR.PA |   AURE.PA |   AUTME.MI |   AV.MI 

Transform: Fill missing values

In [14]:
df_transformed_filled = hf.fillnavalues(df_transformed)

print(df_transformed_filled.tail(3).to_markdown())
print(df_transformed_filled.shape)

| Date                |   A2A.MI |   AAA.PA |   AALB.AS |   AB.PA |   ABCA.PA |   ABEO.PA |   ABI.BR |   ABN.AS |   ABNX.PA |   ABO.BR |   ABT.MI |   ABVX.PA |   AC.PA |   AC5.MI |   ACA.PA |   ACAN.PA |   ACCB.BR |   ACE.MI |   ACKB.BR |   AD.AS |   ADB.MI |   ADOC.PA |   ADP.PA |   ADYEN.AS |   AED.BR |   AEDES.MI |   AEF.MI |   AELIS.PA |   AF.PA |   AFME.PA |   AGFB.BR |   AGN.AS |   AGS.BR |   AI.PA |   AIR.PA |   AJAX.AS |   AKE.PA |   AKOM.PA |   AKW.PA |   AKZA.AS |   ALFEN.AS |   ALK.MI |   ALLFG.AS |   ALO.PA |   ALTA.PA |   ALTR.LS |   ALW.MI |   ALX.AS |   AM.PA |   AMG.AS |   AMP.MI |   AMPLI.PA |   AMUN.PA |   AMUND.AS |   ANIM.MI |   ANTIN.PA |   APAM.AS |   ARAMI.PA |   ARCAD.AS |   AREIT.PA |   ARG.PA |   ARGX.BR |   ARIS.MI |   ARN.MI |   ARTE.PA |   ARTO.PA |   ARVBS.PA |   ARVEN.PA |   ASC.MI |   ASCE.BR |   ASM.AS |   ASML.AS |   ASRNL.AS |   ASY.PA |   ATE.PA |   ATEB.BR |   ATEME.PA |   ATLD.PA |   ATO.PA |   AUB.PA |   AUGR.PA |   AURE.PA |   AUTME.MI |   AV.MI 

Transform: Fill open, high, low, close, volume today's missing values (nan values)

In [15]:
for key in ['df_high', 'df_low', 'df_close', 'df_volume']:
    list_dfs_ohlcv[key].loc[today_date] = np.nan

print(list_dfs_ohlcv['df_high'].tail(3).to_markdown())
print(list_dfs_ohlcv['df_high'].shape)

| Date                |   A2A.MI |   AAA.PA |   AALB.AS |   AB.PA |   ABCA.PA |   ABEO.PA |   ABI.BR |   ABN.AS |   ABNX.PA |   ABO.BR |   ABT.MI |   ABVX.PA |   AC.PA |   AC5.MI |   ACA.PA |   ACAN.PA |   ACCB.BR |   ACE.MI |   ACKB.BR |   AD.AS |   ADB.MI |   ADOC.PA |   ADP.PA |   ADYEN.AS |   AED.BR |   AEDES.MI |   AEF.MI |   AELIS.PA |   AF.PA |   AFME.PA |   AGFB.BR |   AGN.AS |   AGS.BR |   AI.PA |   AIR.PA |   AJAX.AS |   AKE.PA |   AKOM.PA |   AKW.PA |   AKZA.AS |   ALFEN.AS |   ALK.MI |   ALLFG.AS |   ALO.PA |   ALTA.PA |   ALTR.LS |   ALW.MI |   ALX.AS |   AM.PA |   AMG.AS |   AMP.MI |   AMPLI.PA |   AMUN.PA |   AMUND.AS |   ANIM.MI |   ANTIN.PA |   APAM.AS |   ARAMI.PA |   ARCAD.AS |   AREIT.PA |   ARG.PA |   ARGX.BR |   ARIS.MI |   ARN.MI |   ARTE.PA |   ARTO.PA |   ARVBS.PA |   ARVEN.PA |   ASC.MI |   ASCE.BR |   ASM.AS |   ASML.AS |   ASRNL.AS |   ASY.PA |   ATE.PA |   ATEB.BR |   ATEME.PA |   ATLD.PA |   ATO.PA |   AUB.PA |   AUGR.PA |   AURE.PA |   AUTME.MI |   AV.MI 

Transform: Create inputs

In [16]:
df_inputs = inputs.get_inputs(df_transformed_filled, list_dfs_ohlcv)

last_day_index = df_inputs.index.get_level_values(0)[-1] # Get the last value in the top-level index
df_inputs_last_day = df_inputs.loc[last_day_index]

X_inputs_last_day = df_inputs_last_day.values # Convert DataFrame to numpy array

print(df_inputs_last_day.tail(3).to_markdown())

|         |   input_var_past_90d_future_0d |   input_var_past_60d_future_0d |   input_var_past_30d_future_0d |   input_var_past_10d_future_0d |   input_var_past_5d_future_0d |   input_var_past_2d_future_0d |   input_var_past_1d_future_0d |   input_var_past_close_1d |   input_var_past_high_1d |   input_var_past_low_1d |   input_volume_var_90-1d |   input_volume_var_60-1d |   input_volume_var_30-1d |   input_volume_var_10-1d |   input_volume_var_2-1d |   input_volume_var_3-1d |   input_min_var_past_90d |   input_min_var_past_30d |   input_min_var_past_10d |   input_min_var_past_5d |   input_min_var_past_2d |   input_max_var_past_90d |   input_max_var_past_30d |   input_max_var_past_10d |   input_max_var_past_5d |   input_max_var_past_2d |   input_days_since_min_30d |   input_days_since_min_10d |   input_days_since_max_30d |   input_days_since_max_10d |   input_volatility_30d |   input_volatility_10d |   input_volatility_2d |   input_volume_volatility_90-1d |   input_volume_volatility_30-

In [17]:
print(f'shape of X_inputs_last_day (rows -> tested data, columns -> inputs): {X_inputs_last_day.shape}')

shape of X_inputs_last_day (rows -> tested data, columns -> inputs): (853, 52)


In [18]:
scaler = hf.load_object(scaler_path)

X_inputs_last_day_scaled = scaler.transform(X_inputs_last_day)

Load model

In [19]:
model = tf.keras.models.load_model(model_path)

Predict

In [20]:
prediction = model.predict(X_inputs_last_day_scaled)
df_prediction = pd.DataFrame(prediction, columns=['buy_prediction', 'do_not_buy_prediction'], index=df_inputs_last_day.index)
# df_prediction = df_prediction['buy']

df_prediction.drop(columns=['do_not_buy_prediction'], inplace=True)
df_prediction.index.name = 'yahoo_ticker'
df_prediction = df_prediction.sort_values(by='buy_prediction', ascending=False)

print(df_prediction.to_markdown())


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
| yahoo_ticker   |   buy_prediction |
|:---------------|-----------------:|
| VANTI.PA       |        0.943271  |
| AXW.PA         |        0.928252  |
| ABNX.PA        |        0.898648  |
| BOI.PA         |        0.893293  |
| ADOC.PA        |        0.890845  |
| S30.PA         |        0.869466  |
| MRL.MC         |        0.843682  |
| PQ.MI          |        0.841966  |
| APAM.AS        |        0.83481   |
| FNAC.PA        |        0.826562  |
| ALFEN.AS       |        0.808981  |
| SBT.PA         |        0.808408  |
| MMB.PA         |        0.78954   |
| BN.PA          |        0.785265  |
| RAM.LS         |        0.77103   |
| CALT.MI        |        0.770824  |
| ABVX.PA        |        0.763612  |
| MAAT.PA        |        0.762499  |
| PUB.PA         |        0.757679  |
| TEXF.BR        |        0.755364  |
| NACON.PA       |        0.747764  |
| BIG.PA         |        0.744854  |
| LHYFE.PA       |        0.743144  |
| SLB.PA   

In [24]:
df_prediction['bourso_theor_open_price'] = df_prediction.index.map(df_download_today.set_index('ticker')['bourso_theor_open_price'])

df_prediction['buy_volume'] = target_buy_amount / df_prediction['bourso_theor_open_price']
df_prediction['buy_volume'] = df_prediction['buy_volume'].fillna(0)
df_prediction['buy_volume'] = np.ceil(df_prediction['buy_volume']).astype(int)

df_prediction['sell_price'] = df_prediction['bourso_theor_open_price'] * loss_limit

print(df_prediction.head(30).to_markdown())

| yahoo_ticker   |   buy_prediction |   bourso_theor_open_price |   buy_volume |   sell_price |
|:---------------|-----------------:|--------------------------:|-------------:|-------------:|
| VANTI.PA       |         0.943271 |                     0.114 |        18422 |     0.113658 |
| AXW.PA         |         0.928252 |                    21.6   |           98 |    21.5352   |
| ABNX.PA        |         0.898648 |                     1.19  |         1765 |     1.18643  |
| BOI.PA         |         0.893293 |                    31.6   |           67 |    31.5052   |
| ADOC.PA        |         0.890845 |                     5.79  |          363 |     5.77263  |
| S30.PA         |         0.869466 |                     1.59  |         1321 |     1.58523  |
| MRL.MC         |         0.843682 |                    10     |          210 |     9.97     |
| PQ.MI          |         0.841966 |                     2.18  |          964 |     2.17346  |
| APAM.AS        |         0.83481  |   

In [25]:
df_prediction['bourso'] = df_prediction.index.map(df_tickers.set_index('yahoo')['bourso'])
df_prediction['euronext'] = df_prediction.index.map(df_tickers.set_index('yahoo')['euronext'])
df_prediction['fortuneo'] = df_prediction.index.map(df_tickers.set_index('yahoo')['fortuneo'])

df_prediction['bourso'] = base_url_bourso + df_prediction['bourso']
df_prediction['euronext'] = base_url_euronext + df_prediction['euronext']
df_prediction['fortuneo'] = base_url_fortuneo + df_prediction['fortuneo']

print(df_prediction.head(30).to_markdown())

| yahoo_ticker   |   buy_prediction |   bourso_theor_open_price |   buy_volume |   sell_price | bourso                                     | euronext                                                        | fortuneo                                                                              |
|:---------------|-----------------:|--------------------------:|-------------:|-------------:|:-------------------------------------------|:----------------------------------------------------------------|:--------------------------------------------------------------------------------------|
| VANTI.PA       |         0.943271 |                     0.114 |        18422 |     0.113658 | https://www.boursorama.com/cours/1rPVANTI  | https://live.euronext.com/en/product/equities/FR0013505062-XPAR | https://bourse.fortuneo.fr/actions/cours-vantiva-VANTI-FR0013505062-23                |
| AXW.PA         |         0.928252 |                    21.6   |           98 |    21.5352   | https://www.boursor

In [26]:
df_prediction_filtered = df_prediction[df_prediction['buy_prediction'] > cumulated_probs_target]
print(df_prediction_filtered.to_markdown())

| yahoo_ticker   |   buy_prediction |   bourso_theor_open_price |   buy_volume |   sell_price | bourso                                    | euronext                                                        | fortuneo                                                                     |
|:---------------|-----------------:|--------------------------:|-------------:|-------------:|:------------------------------------------|:----------------------------------------------------------------|:-----------------------------------------------------------------------------|
| VANTI.PA       |         0.943271 |                     0.114 |        18422 |     0.113658 | https://www.boursorama.com/cours/1rPVANTI | https://live.euronext.com/en/product/equities/FR0013505062-XPAR | https://bourse.fortuneo.fr/actions/cours-vantiva-VANTI-FR0013505062-23       |
| AXW.PA         |         0.928252 |                    21.6   |           98 |    21.5352   | https://www.boursorama.com/cours/1rPAXW   | https

Theoretical opening price confirmation using Euronext

In [46]:
df_confirmation = pd.DataFrame({})
df_confirmation.index = df_prediction_filtered.index

mapping_dict = dict(zip(df_tickers['yahoo'], df_tickers['euronext']))
df_confirmation['euronext_ticker'] = df_confirmation.index.map(mapping_dict)

euronext_tickers = df_confirmation['euronext_ticker'].to_list()
df_euronext = euronext.get_theoretical_opening_prices(euronext_tickers)

print(df_euronext)

Error fetching price for FR0013505062-XPAR: Message: invalid selector
from javascript error: {"status":32,"value":"Unable to locate an element with the xpath expression //span[contains(text(), 'Cours Théorique d'Ouverture')]/following-sibling::span because of the following error:\nSyntaxError: Failed to execute 'evaluate' on 'Document': The string '//span[contains(text(), 'Cours Théorique d'Ouverture')]/following-sibling::span' is not a valid XPath expression."}
  (Session info: chrome-headless-shell=126.0.6478.183); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
0   chromedriver                        0x0000000105092a0c chromedriver + 4385292
1   chromedriver                        0x000000010508b318 chromedriver + 4354840
2   chromedriver                        0x0000000104ca8b0c chromedriver + 281356
3   chromedriver                        0x0000000104cacfd8 chromedriver + 

In [47]:
mapping_dict = dict(zip(df_tickers['euronext'], df_tickers['yahoo']))
df_euronext['ticker'] = df_euronext['ticker'].map(mapping_dict)

print(df_euronext)


     ticker euronext_theor_open_price
0  VANTI.PA                      None
1    AXW.PA                      None
2   ABNX.PA                      None
3    BOI.PA                      None
4   ADOC.PA                      None
5    S30.PA                      None


In [61]:
mapping_dict = dict(zip(df_euronext['ticker'], df_prediction_filtered.index))
df_prediction_filtered.loc[:,'euronext_theor_open_price'] = df_euronext['euronext_theor_open_price'].map(mapping_dict)

cols = list(df_prediction_filtered.columns)
cols.insert(2, cols.pop(cols.index('euronext_theor_open_price')))
df_prediction_filtered = df_prediction_filtered[cols]

print(df_prediction_filtered.to_markdown())

| yahoo_ticker   |   buy_prediction |   bourso_theor_open_price |   euronext_theor_open_price |   buy_volume |   sell_price | bourso                                    | euronext                                                        | fortuneo                                                                     |
|:---------------|-----------------:|--------------------------:|----------------------------:|-------------:|-------------:|:------------------------------------------|:----------------------------------------------------------------|:-----------------------------------------------------------------------------|
| VANTI.PA       |         0.943271 |                     0.114 |                         nan |        18422 |     0.113658 | https://www.boursorama.com/cours/1rPVANTI | https://live.euronext.com/en/product/equities/FR0013505062-XPAR | https://bourse.fortuneo.fr/actions/cours-vantiva-VANTI-FR0013505062-23       |
| AXW.PA         |         0.928252 |                  

Evaluate

In [ ]:
# df_open = list_dfs_ohlcv['df_open']
# df_close = list_dfs_ohlcv['df_close']

# df_open_last_day = df_open.iloc[-1]
# df_close_last_day = df_close.iloc[-1]

# df_real = pd.DataFrame()
# df_real['variation'] = df_close_last_day / df_open_last_day

In [ ]:
# df_low = list_dfs_ohlcv['df_low']
# df_low_last_day = df_low.loc[last_day_index]
# df_real['lowest_variation'] = df_low_last_day / df_open_last_day

# df_real['profit'] = df_real['variation']
# df_real.loc[df_real['lowest_variation'] < loss_limit, 'profit'] = loss_limit
# df_real['profit'] *= hf.get_fee_coef(fee)

In [ ]:
# df_evaluation = pd.concat([df_prediction, df_real], axis='columns')
# print(df_evaluation)

In [ ]:
# df_filtered_evaluation = df_evaluation[df_evaluation['buy'] > cumulated_probs_target]

# count = df_filtered_evaluation['profit'].count()
# average = df_filtered_evaluation['profit'].mean()
# median = df_filtered_evaluation['profit'].median()

# print(f'count: {count}')
# print(f'average profit: {average}')
# print(f'median profit: {median}')

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 7))
# plt.scatter(x=df_filtered_evaluation['buy'], y=df_filtered_evaluation['variation'], color='blue', alpha=0.3, s=5)
# plt.title('variation vs buy prediction')
# plt.xlabel('buy prediction')
# plt.ylabel('real variation')
# plt.grid(True)
# plt.show()